# Logit-SUE: an illustration of fixed point formulation with MSA as the solution algorithm.

In [1]:
using DelimitedFiles:readdlm

## Problem setting.

<img src="data/Caliper_net.png" width="800">

In [2]:
netFile = "data/network.csv"
csvdata = readdlm(netFile, ',', header=true)
data = csvdata[1]
header = csvdata[2]

1×10 Matrix{AbstractString}:
 "tail"  "head"  "cap"  "length"  "fft"  …  "speedLimit"  "toll"  "link_type"

In [3]:
display(header)
display(data)

1×10 Matrix{AbstractString}:
 "tail"  "head"  "cap"  "length"  "fft"  …  "speedLimit"  "toll"  "link_type"

7×10 Matrix{Float64}:
 1.0  2.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 1.0  4.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 2.0  3.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 2.0  5.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 3.0  6.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 4.0  5.0  50.0  20.0  10.0  0.15  4.0  60.0  4.0  1.0
 5.0  6.0  50.0  20.0  10.0  0.15  4.0  60.0  2.0  1.0

In [6]:
links = []
for l in zip(round.(Int, data[:,1]), round.(Int, data[:,2]))
    push!(links, l)
end
cap = data[:, 3]
fft = data[:, 5]
alpha = data[:, 6]
beta = data[:, 7]
dem = 100.0

100.0

In [4]:
# BPR function
function BPR(cap, fft, alpha, beta,  flow)
    return fft * (1 + alpha * (flow/cap) ^ beta)
end

BPR (generic function with 1 method)

In [5]:
# Logit probability
function Logit(cost)
    theta = 1.0
    util = -cost
    return exp.(theta * util) / sum(exp.(theta * util))
end

Logit (generic function with 1 method)

In [7]:
# link-path incidence matrix (a 7 x 3 matrix)
display(links)
Δ = Matrix([
    1 1 0;
    0 0 1;
    1 0 0;
    0 1 0;
    1 0 0;
    0 0 1;
    0 1 1
    ])

7-element Vector{Any}:
 (1, 2)
 (1, 4)
 (2, 3)
 (2, 5)
 (3, 6)
 (4, 5)
 (5, 6)

7×3 Matrix{Int64}:
 1  1  0
 0  0  1
 1  0  0
 0  1  0
 1  0  0
 0  0  1
 0  1  1

In [8]:
# path flow vector (a temporary value for illustration)
f = [30.0, 30.0, 40.0]

3-element Vector{Float64}:
 30.0
 30.0
 40.0

In [9]:
# link flow vector
x = Δ * f

7-element Vector{Float64}:
 60.0
 40.0
 30.0
 30.0
 30.0
 40.0
 70.0

In [10]:
# link time
t = BPR.(cap, fft, alpha, beta, x)

7-element Vector{Float64}:
 26.2208
 21.2288
 20.388799999999996
 20.388799999999996
 20.388799999999996
 10.6144
 15.7624

In [11]:
# path time
c = Δ' * t

3-element Vector{Float64}:
 66.9984
 62.372
 47.605599999999995

In [12]:
f = Logit(c) * dem

3-element Vector{Float64}:
  3.7828036716149295e-7
  3.863970418983494e-5
 99.99996098201545

## Formulation
It is a fixed point problem with respect to path flow. 
$$
\mathbf{f} = F(\mathbf{f}) \Rightarrow \mathbf{f = q * Pr(-\Delta^{T}t(\Delta f))}
$$

Alternatively, we can also write it as a fixed point problem with respect to link flow.
$$
\mathbf{x} = G(\mathbf{x}) \Rightarrow \mathbf{x = \Delta q * Pr(-\Delta^{T}t(x))}
$$

We use MSA method to solve this fixed point problem with respect to path flow. 

In [13]:
# intial value
f = [100; 0; 0]

3-element Vector{Int64}:
 100
   0
   0

In [14]:
# iteration 1
iter = 1
x = Δ * f
t = BPR.(cap, fft, alpha, beta, x)
c = Δ' * t
f_aux = Logit(c) * dem
f = (f_aux + iter * f) / (1 + iter)

3-element Vector{Float64}:
 50.0
  3.23511746282273e-24
 50.0

In [15]:
# iteration 2
iter = 2
x = Δ * f
t = BPR.(cap, fft, alpha, beta, x)
c = Δ' * t
f_aux = Logit(c) * dem
f = (f_aux + iter * f) / (1 + iter)

3-element Vector{Float64}:
 33.33333333675326
  0.006780899267811179
 66.65988576397892

In [16]:
# iteration 3
iter = 3
x = Δ * f
t = BPR.(cap, fft, alpha, beta, x)
c = Δ' * t
f_aux = Logit(c) * dem
f = (f_aux + iter * f) / (1 + iter)

3-element Vector{Float64}:
 25.038640353662107
 24.317479701630738
 50.643879944707145

In [17]:
# iteration 4
iter = 4
x = Δ * f
t = BPR.(cap, fft, alpha, beta, x)
c = Δ' * t
f_aux = Logit(c) * dem
f = (f_aux + iter * f) / (1 + iter)

3-element Vector{Float64}:
 20.031277190680107
 19.459053314183905
 60.509669495135974

In [18]:
# iteration 5
iter = 5
x = Δ * f
t = BPR.(cap, fft, alpha, beta, x)
c = Δ' * t
f_aux = Logit(c) * dem
f = (f_aux + iter * f) / (1 + iter)

3-element Vector{Float64}:
 18.608477833206802
 18.72720868604431
 62.664313480748866

In [19]:
# Let's do it for many loops.
f = [100; 0; 0]
f_aux = []
for iter in 1:1000
    f_aux = dem * Logit(Δ' * BPR.(cap, fft, alpha, beta, Δ * f))
    f = (f_aux  + iter * f) / (1 + iter)
end
println("f_aux = $f_aux")
println("f     = $f")
println("x     = $(Δ*f)")

f_aux = [19.505688245072662, 19.411985387194054, 61.08232636773327]
f     = [19.505688245104935, 19.411985387147734, 61.08232636774742]
x     = [38.91767363225267, 61.08232636774742, 19.505688245104935, 19.411985387147734, 19.505688245104935, 61.08232636774742, 80.49431175489515]


In [20]:
# Let's also do it link flow fixed point problem.
# Initial value
f = [0; 100; 0]
x = Δ*f
x_aux = []
for iter in 1:1000
    x_aux = Δ * dem * Logit(Δ' * BPR.(cap, fft, alpha, beta, x))
    x = (x_aux  + iter * x) / (1 + iter)
end
println("x_aux = $x_aux")
println("x     = $x")
println("f     = $(Δ'*x). Error: we cannot know path flow from link flows.")

x_aux = [38.9176736322585, 61.08232636774149, 19.50568824508956, 19.41198538716894, 19.50568824508956, 61.08232636774149, 80.49431175491043]
x     = [38.9176736322545, 61.08232636774574, 19.505688245100558, 19.411985387153848, 19.505688245100558, 61.08232636774574, 80.49431175489903]
f     = [77.9290501224556, 138.82397077430738, 202.65896449039053]. Error: we cannot know path flow from link flows.

## The auxiliary solution converges to the current solution.